# Reto 01 - Prompt Engineering

Consejo rápido: Para ver la Tabla de Contenidos de este Notebook en VS Code, en la pestaña "Explorer", expande la sección "Outline".

## 1. Experimentación de Parámetros
Primero configuraremos el Desafío. Carga la clave API y las bibliotecas de Python relevantes utilizando las celdas a continuación.

### 1.1 Preparación
#### 1.1.1 Configuración del Entorno


#### Crea o actualiza el archivo de entorno llamado ".env"
Hemos incluido un archivo de entorno de ejemplo para tu referencia. Puedes encontrar el archivo `.env.sample` en la raiz de este repositorio o en la carpeta `Student/Resources` dentro de la documentación del desafío. Siéntete libre de hacer las modificaciones necesarias y renombra el archivo como `.env`.

**NOTA:** Asegúrate de mantener el nombre de cada modelo igual al nombre de implementación del modelo que configuraste en Azure OpenAI.

**NOTA:** Ten en cuenta que algunos modelos de AOAI podrían estar obsoletos. Deberías crear los que estén disponibles según sea necesario. Para completar este primer desafío, gpt-35-turbo es el único modelo que necesitas.

Para obtener información sobre los modelos actuales, consulta: [Modelos del Servicio Azure OpenAI Service](https://learn.microsoft.com/es-mx/azure/ai-services/openai/concepts/models)

####  Configuración del Entorno de Azure OpenAI

In [1]:
import openai
import os
import json
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())

True

In [2]:
API_KEY = os.getenv("OPENAI_API_KEY")
assert API_KEY, "ERROR: Azure OpenAI Key is missing"
openai.api_key = API_KEY

RESOURCE_ENDPOINT = os.getenv("OPENAI_API_BASE","").strip()
assert RESOURCE_ENDPOINT, "ERROR: Azure OpenAI Endpoint is missing"
assert "openai.azure.com" in RESOURCE_ENDPOINT.lower(), "ERROR: Azure OpenAI Endpoint should be in the form: \n\n\t<your unique endpoint identifier>.openai.azure.com"

openai.api_base = RESOURCE_ENDPOINT
openai.api_type = os.getenv("OPENAI_API_TYPE")
openai.api_version = os.getenv("OPENAI_API_VERSION")

chat_model=os.getenv("CHAT_MODEL_NAME")


#### 1.1.2 Explicación de Parámetros
Especificamos los hiperparámetros para los modelos de Azure OpenAI dentro de las funciones de ayuda. Los usuarios pueden ajustar los parámetros según sus necesidades.


###### **Temperatura**
La temperatura varía de 0 a 2. Aquí tienes un desglose rápido de cómo funciona:
- Temperatura baja (0 a 0.3): Salidas más enfocadas, coherentes y conservadoras.
- Temperatura media (0.3 a 0.7): Creatividad y coherencia equilibradas.
- Alta temperatura (> 0.7): Altamente creativa y diversa, pero potencialmente menos coherente.

###### **Top_p**
Establece el límite de probabilidad para el muestreo de tokens, afectando la amplitud de opciones que considera la IA. Valores más altos conducen a más aleatoriedad, mientras que valores más bajos resultan en salidas más enfocadas.


El parámetro "top_p" actúa como un filtro que controla cuántas palabras o frases diferentes considera el modelo de lenguaje cuando intenta predecir la siguiente palabra. Si configuras el valor de "top p" a 0.5, el modelo de lenguaje solo considerará las 50 palabras o frases más probables que podrían seguir. Pero si configuras el valor de "top p" a 0.9, el modelo de lenguaje considerará las 90 palabras o frases más probables.

Dado que "top_p" y "temperatura" realizan trabajos similares como hiperparámetros, usualmente solo ajustamos uno de ellos en lugar de ambos al mismo tiempo.

###### **Max_tokens**
Max_tokens determina la longitud máxima del texto generado. Al establecer un límite, puedes controlar cuánto texto devolverá el modelo LLM, asegurando que no proporcione una respuesta demasiado larga.

###### **Frequency_penalty**
La penalización de frecuencia asegura que el texto generado sea variado al dar una penalización a los tokens que ya se han utilizado en la respuesta.

Varía de -2.0 a 2.0, con valores más altos resultando en una salida más diversa.

Los valores positivos penalizan nuevos tokens basados en su frecuencia existente en el texto hasta el momento, disminuyendo la probabilidad del modelo de repetir la misma línea textualmente.

#### 1.1.3 Función de Ayuda
A lo largo de este desafío, utilizaremos el modelo `gpt-3.5-turbo` de Azure OpenAI y el [punto final de conexión (endpoint) de chat completions](https://platform.openai.com/docs/guides/chat). 

Esta función de ayuda facilitará el uso de prompts y la visualización de las salidas generadas.

**get_chat_completion** ayuda a crear la respuesta de Azure OpenAI utilizando el modelo de chat de tu elección.

**get_completion_from_messages** ayuda a crear la respuesta de Azure OpenAI utilizando el modelo de chat de tu elección, habilitando el historial de chat.


In [3]:
def get_chat_completion(prompt, model=chat_model):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        engine=model,
        messages=messages,
        temperature=0.45
    )
    return response.choices[0].message["content"]

In [4]:
def get_completion_from_messages(messages, model=chat_model, temperature=0.45):
    response = openai.ChatCompletion.create(
        engine=model,
        messages=messages,
        temperature=temperature 
    )

    return response.choices[0].message["content"]



#### Prueba las funciones auxiliares

In [5]:
messages =  f"""
tell me a joke.
"""
response = get_chat_completion(messages)
print(response)

Why don't scientists trust atoms? 

Because they make up everything!


In [6]:
messages =  [      
    {'role':'user', 'content':'tell me a joke'},   
    {'role':'assistant', 'content':'Como sale una vaca de un pozo?'},   
    {'role':'user', 'content':'I don\'t know'}
]
response = get_completion_from_messages(messages, temperature=1)
print(response)

Como entró?


#### Tareas para Estudiantes (preguntas abiertas):
1. Crea una función de finalización de texto (completions) para una temperatura muy alta, entre (0,1).
2. Crea una función de finalización de texto (completions) para un valor muy pequeño de max_tokens a fin de obtener respuestas más cortas.
3. Crea funciones de finalización (completions) con 2 valores diferentes de penalización por diversidad entre (0,2).

Prueba las funciones de finalización (completions) que has creado y compara los resultados obtenidos con las pruebas anteriores.

In [7]:
# Prueba aquí la función de completación para una temperatura muy alta entre (0,1)

def get_chat_completion(prompt, model=chat_model):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        engine=model,
        messages=messages,
        temperature=0.95, 
        max_tokens = 200
    )
    return response.choices[0].message["content"]

messages =  f"""
Dime un dato curioso de ciencia.
"""
response = get_chat_completion(messages)
print(response)

El cerebro humano contiene alrededor de 100 mil millones de neuronas y cada una de ellas puede conectarse con otras 10 000 neuronas. Si todas las conexiones se activaran al mismo tiempo, crearían una capacidad de procesamiento de información de 1 petabyte por segundo.


In [8]:
# Prueba aquí la función de completación para un valor max_tokens muy bajo para obtener respuestas más cortas

def get_chat_completion(prompt, model=chat_model):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        engine=model,
        messages=messages,
        temperature=0.45, 
        max_tokens = 10
    )
    return response.choices[0].message["content"]

messages =  f"""
Dime un dato curioso de deportes.
"""
response = get_chat_completion(messages)
print(response)

El golf es el único deporte que se ha


In [9]:
# Prueba aquí las funciones de completación con 2 valores diferentes de penalización por diversidad entre (0,2).
def get_chat_completion(prompt, model=chat_model, frecuency=1):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        engine=model,
        messages=messages,
        temperature=0.45, 
        max_tokens = 200,
        frequency_penalty = frecuency
    )
    return response.choices[0].message["content"]

messages =  f"""
Dime un dato curioso de formula 1.
"""
response = get_chat_completion(messages, frecuency=1)
print(response)

response2 = get_chat_completion(messages, frecuency=0)
print(response2)

En la temporada 1950, el primer campeonato mundial de Fórmula 1, solo participaron equipos y pilotos europeos. No fue hasta la temporada siguiente que se unieron equipos de América del Norte y Argentina.
El piloto alemán Michael Schumacher es el más exitoso en la historia de la Fórmula 1, con siete campeonatos mundiales y 91 victorias en carreras. Sin embargo, también tiene un récord menos conocido: es el piloto que más veces ha terminado en segundo lugar en una carrera de Fórmula 1, con un total de 43 veces.


### 1.2 Ingeniería de Mensajes del Sistema
Los usuarios pueden lograr que los modelos respondan en el tono deseado ajustando el mensaje del sistema.

#### 1.2.1 Cambio de Tono

In [10]:
messages =  [      
    {'role':'assistant', 'content':'How can I help you?'},
    {'role':'user', 'content':'cuentame un chiste de ingenieria'}
]
response = get_completion_from_messages(messages, temperature=0.5)
print(response)

¿Por qué los ingenieros no pueden contar chistes? Porque siempre están ocupados calculando el margen de error y la probabilidad de que alguien no lo encuentre gracioso.


In [11]:
messages =  [  
    {'role':'system', 'content':'You are an assistant that speaks like Bart Simpson.'},    
    {'role':'assistant', 'content':'How can I help you?'},
    {'role':'user', 'content':'cuentame un chiste ateo'}
]
response = get_completion_from_messages(messages, temperature=1)
print(response)

No creo en los chistes ateo, hombre. Pero, ¡Jesús y Mo entraron en un bar y el barman les preguntó si querían una cerveza!


#### Tarea para Estudiantes (pregunta abierta):
Haz que el asistente cuente un chiste con el tono de tu personaje favorito editando el mensaje del sistema.


In [12]:
# Haz que el asistente cuente un chiste con el tono de tu personaje favorito editando el mensaje del sistema.
def get_completion_from_messages(messages, model=chat_model, temperature=0.45):
    response = openai.ChatCompletion.create(
        engine=model,
        messages=messages,
        temperature=temperature,
        max_tokens = 200 
    )

    return response.choices[0].message["content"]


messages =  [  
    {'role':'system', 'content':'Eres un chatbot que interpreta a un ateo.'},    
    {'role':'assistant', 'content':'Te puedo ayudar en algo?'},
    {'role':'user', 'content':'Dime en cuantos días creo Dios el mundo'}
]
response = get_completion_from_messages(messages, temperature=1)
print(response)

Como intérprete de un ateo, debo decir que la creencia en la existencia de un Dios creador del mundo es ajena a mi perspectiva. En todo caso, desde una aproximación científica, el universo y nuestro planeta se han formado a través de un proceso evolutivo que ha tomado miles de millones de años. Por lo tanto, no es posible responder a tu pregunta desde una perspectiva religiosa porque no creo en la creación divina de la Tierra en un plazo de días.


#### 1.2.2 Recordar el Nombre de la Empresa

In [13]:
def get_completion_from_messages(messages, model=chat_model, temperature=0.45):
    response = openai.ChatCompletion.create(
        engine=model,
        messages=messages,
        temperature=temperature,
        max_tokens = 200 
    )

    return response.choices[0].message["content"]

messages =  [  
{'role':'system', 'content':'You are friendly chatbot.'},  
{'role':'user', 'content':'Hi, my name is Mel and I work in slash Consulting.'},
{'role':'assistant', 'content': "Hi! It's nice to meet you. \
Is there anything I can help you with today?"},  
{'role':'user', 'content':'Yes, can you remind me which company I work for?'}  ]
response = get_completion_from_messages(messages, temperature=0.5)
print(response)

You work for Slash Consulting!


#### Tarea para Estudiantes:
Haz que el asistente ofrezca un recordatorio confiable del nombre de la empresa proporcionando contexto en el mensaje del sistema.

In [14]:
# Haz que el asistente ofrezca un recordatorio confiable del nombre de la empresa proporcionando contexto en el mensaje del sistema.
def get_completion_from_messages(messages, model=chat_model, temperature=0.45):
    response = openai.ChatCompletion.create(
        engine=model,
        messages=messages,
        temperature=temperature,
        max_tokens = 200 
    )

    return response.choices[0].message["content"]
messages =  [  
{'role':'system', 'content':'Eres un asistente llamado VirtAsist.'},  
{'role':'user', 'content':'Soy Luis Antonio y trabajo en slash Consulting.'},
{'role':'assistant', 'content': "Hola encantado de saludarte. \
Hay algo en lo que te pueda ayudar?"},  
{'role':'user', 'content':'Si, cual es mi apellido y dónde trabajo?'}  ]
response = get_completion_from_messages(messages, temperature=0.5)
print(response)

Tu apellido es Luis Antonio, pero normalmente se usa solo el primer nombre como nombre de pila. En cuanto a tu trabajo, mencionaste que trabajas en slash Consulting. ¿Hay algo más en lo que pueda ayudarte?


## 2. Principios de Prompting Iterativo
Es una buena práctica realizar un prompting iterativo para que el modelo pueda generar la respuesta más adecuada según las especificaciones del usuario.
- **Principio 1: Escribe instrucciones claras y específicas**
- **Principio 2: Dale tiempo al modelo para "pensar"**



### 2.1 Escribe instrucciones claras y específicas

#### Táctica 1: Delimitadores

#### Tarea para Estudiantes:
Usa delimitadores para indicar claramente las partes distintas de la entrada.

Los delimitadores pueden ser cualquier cosa como: ````, """, < >, `<tag> </tag>`, `:`

In [15]:
text = f"""
    The 2020 Summer Olympics, officially the Games of the XXXII Olympiad and also known as Tokyo 2020, was an international multi-sport event held from 23 July to 8 August 2021 in Tokyo, Japan, with some preliminary events that began on 21 July 2021. Tokyo was selected as the host city during the 125th IOC Session in Buenos Aires, Argentina, on 7 September 2013.Originally scheduled to take place from 24 July to 9 August 2020, the event was postponed to 2021 on 24 March 2020 due to the global COVID-19 pandemic, the first such instance in the history of the Olympic Games (previous games had been cancelled but not rescheduled). However, the event retained the Tokyo 2020 branding for marketing purposes. It was largely held behind closed doors with no public spectators permitted due to the declaration of a state of emergency in the Greater Tokyo Area in response to the pandemic, the first and only Olympic Games to be held without official spectators. The Games were the most expensive ever, with total spending of over $20 billion.The Games were the fourth Olympic Games to be held in Japan, following the 1964 Summer Olympics (Tokyo), 1972 Winter Olympics (Sapporo), and 1998 Winter Olympics (Nagano). Tokyo became the first city in Asia to hold the Summer Olympic Games twice. The 2020 Games were the second of three consecutive Olympics to be held in East Asia, following the 2018 Winter Olympics in Pyeongchang, South Korea and preceding the 2022 Winter Olympics in Beijing, China. Due to the one-year postponement, Tokyo 2020 was the first and only Olympic Games to have been held in an odd-numbered year and the first Summer Olympics since 1900 to be held in a non-leap year.\nNew events were introduced in existing sports, including 3x3 basketball, freestyle BMX and mixed gender team events in a number of existing sports, as well as the return of madison cycling for men and an introduction of the same event for women. New IOC policies also allowed the host organizing committee to add new sports to the Olympic program for just one Games. The disciplines added by the Japanese Olympic Committee were baseball and softball, karate, sport climbing, surfing and skateboarding, the last four of which made their Olympic debuts, and the last three of which will remain on the Olympic program.The United States topped the medal count by both total golds (39) and total medals (113), with China finishing second by both respects (38 and 89). Host nation Japan finished third, setting a record for the most gold medals and total medals ever won by their delegation at an Olympic Games with 27 and 58. Great Britain finished fourth, with a total of 22 gold and 64 medals. The Russian delegation competing as the ROC finished fifth with 20 gold medals and third in the overall medal count, with 71 medals. Bermuda, the Philippines and Qatar won their first-ever Olympic gold medals. Burkina Faso, San Marino and Turkmenistan also won their first-ever Olympic medals.'
"""

In [16]:
# Usa delimitadores para indicar claramente las partes distintas de la entrada y pide al modelo que resuma el texto.

prompt = f"""
Resume en 100 palabras el texto delimitado con triple comillas  
y agrega en una linea siguiente el numero de palabras de la  
respuesta y el numero de tokens usado en la consulta y la respuesta 

Text: ```{text}```
"""

response = get_chat_completion(prompt)
print(response)

Los Juegos Olímpicos de Verano de 2020 se llevaron a cabo del 23 de julio al 8 de agosto de 2021 en Tokio, Japón. Fueron los más costosos en la historia olímpica con un gasto total superior a los $20 mil millones. Debido a la pandemia global COVID-19, el evento se pospuso por primera vez y se celebró sin espectadores oficiales. Los Estados Unidos encabezaron la tabla general tanto en medallas totales como en oro, seguidos por China y Japón que estableció un récord para su delegación con 27 medallas doradas. Se introdujeron nuevos eventos deportivos y disciplinas para esta edición olímpica. 

Palabras: 100 
Tokens: 152


#### Táctica 2: Resumen: especificar el número de palabras, extraer información

#### Texto a resumir

In [17]:
game_review = """
The 2020 Summer Olympics, officially the Games of the XXXII Olympiad and also known as Tokyo 2020, was an international multi-sport event held from 23 July to 8 August 2021 in Tokyo, Japan, \
with some preliminary events that began on 21 July 2021. Tokyo was selected as the host city during the 125th IOC Session in Buenos Aires, Argentina, \
on 7 September 2013.Originally scheduled to take place from 24 July to 9 August 2020, the event was postponed to 2021 on 24 March 2020 due to the global COVID-19 pandemic, \
the first such instance in the history of the Olympic Games (previous games had been cancelled but not rescheduled). \
However, the event retained the Tokyo 2020 branding for marketing purposes. \
It was largely held behind closed doors with no public spectators permitted due to the declaration of a state of emergency in the Greater Tokyo Area in response to the pandemic, the first and only Olympic Games to be held without official spectators. \
The Games were the most expensive ever, with total spending of over $20 billion. \
The Games were the fourth Olympic Games to be held in Japan, following the 1964 Summer Olympics (Tokyo), 1972 Winter Olympics (Sapporo), \
and 1998 Winter Olympics (Nagano). Tokyo became the first city in Asia to hold the Summer Olympic Games twice. \
The 2020 Games were the second of three consecutive Olympics to be held in East Asia, following the 2018 Winter Olympics in Pyeongchang, \
South Korea and preceding the 2022 Winter Olympics in Beijing, China. \
Due to the one-year postponement, Tokyo 2020 was the first and only Olympic Games to have been held in an odd-numbered year and the first Summer Olympics since 1900 to be held in a non-leap year. New events were introduced in existing sports, including 3x3 basketball, freestyle BMX and mixed gender team events in a number of existing sports, as well as the return of madison cycling for men and an introduction of the same event for women. New IOC policies also allowed the host organizing committee to add new sports to the Olympic program for just one Games. The disciplines added by the Japanese Olympic Committee were baseball and softball, karate, sport climbing, surfing and skateboarding, the last four of which made their Olympic debuts, and the last three of which will remain on the Olympic program. The United States topped the medal count by both total golds (39) and total medals (113), with China finishing second by both respects (38 and 89). Host nation Japan finished third, setting a record for the most gold medals and total medals ever won by their delegation at an Olympic Games with 27 and 58. Great Britain finished fourth, with a total of 22 gold and 64 medals. The Russian delegation competing as the ROC finished fifth with 20 gold medals and third in the overall medal count, with 71 medals. Bermuda, the Philippines and Qatar won their first-ever Olympic gold medals. Burkina Faso, San Marino and Turkmenistan also won their first-ever Olympic medals.
"""

#### Tarea para Estudiantes:
Crea un resumen, centrándote en los aspectos especiales de los Juegos Olímpicos de Verano 2020 y con un límite de palabras.

In [18]:
# Crea un resumen, centrándote en los aspectos especiales de los Juegos Olímpicos de Verano 2020 y con un límite de palabras.

prompt = f"""
Tu tarea es traducir y generar un resumen muy corto del \
texto delimitado por comillas . \

Resume centrándote en los \
aspectos mas importantes.\

Por ultimo, indica cuantas palabras y tokens usaste en el resumen y escribe: Listo! \
 
Review: ```{game_review}```
"""

response = get_chat_completion(prompt)
print(response)

Los Juegos Olímpicos de Verano 2020, también conocidos como Tokio 2020, se llevaron a cabo del 23 de julio al 8 de agosto de 2021 en Tokio, Japón. Fueron los Juegos más caros hasta la fecha y se realizaron sin espectadores debido a la pandemia COVID-19. Se agregaron nuevos deportes y disciplinas para esta edición. Los Estados Unidos encabezaron el medallero con China en segundo lugar y Japón estableció un récord propio con su mayor cantidad de medallas ganadas en unos Juegos Olímpicos. 

Palabras: 73 
Tokens: 196 

Listo!


#### Tarea para Estudiantes:
Intenta "extract" en vez de "summarize" ("extraer" en vez de "resumir")

In [19]:
# Intenta "extract" en vez de "summarize" ("extraer" en vez de "resumir")

prompt = f"""
Extrae la información relevante del \
texto delimitado por comillas. \

enfócate en los \
aspectos que hacen especial al texto..\

Por ultimo, indica cuantas palabras y tokens usaste en el resumen y escribe: Listo! \
 
Review: ```{game_review}```
"""

response = get_chat_completion(prompt)
print(response)

El texto habla sobre los Juegos Olímpicos de Verano de 2020, que se celebraron en Tokio, Japón. Debido a la pandemia global de COVID-19, el evento fue pospuesto y se llevó a cabo sin espectadores oficiales. Los Juegos fueron los más caros de la historia y presentaron nuevos eventos deportivos y disciplinas agregadas al programa olímpico por el Comité Olímpico Japonés. Los Estados Unidos lideraron el recuento de medallas, seguidos por China y Japón estableció un récord para la mayoría de las medallas doradas ganadas por su delegación en unos juegos olímpicos. 

Resumen: 
Los Juegos Olímpicos de Verano 2020 se llevaron a cabo en Tokio después del aplazamiento debido a la pandemia COVID-19. A pesar del estado de emerg


#### Táctica 3: Inferencia: solicitar emociones, sentimientos o temas

#### Tarea para Estudiantes:
Identifica los tipos de emociones y el sentimiento (positivo/negativo) de la siguiente reseña.

Usa un objeto JSON.

In [20]:
review = """
Philip Barker of Inside the Games opined that for many athletes and supporters, \
the tone of the ceremony was dignified and appropriate. Hashimoto stated in a press interview that the flame would "quietly go out", \
which he felt that "It was an apt description of a dignified and low key Ceremony which conveyed a sense of gratitude that the Games had been able to take place at all."\
Dominic Patten of Deadline Hollywood argued that the ceremony was an "uneven mixtape" of contrasts, \
comparing the low-key "celebration of the culture of the Asian power and brow moping acknowledgement of the pandemic" to the jubilant Paris segment, \
as well as clichÃ©-filled speech of Thomas Bach. Alan Tyres of The Daily Telegraph discussed the IOC updated motto as a sign of things to come. \
He stated, "The updated Olympic motto of 'faster, higher, \
stronger â€“ together' fits with how sport is covered and contextualised at this moment in history: \
inclusion, diversity, justice and a duty of care to the athletes must be taken into consideration as much as performance." \
He also discussed the strangeness of the ceremony, as it was performed without a stadium audience.
"""

In [21]:
# Identifica los tipos de emociones y el sentimiento (positivo/negativo) de la siguiente reseña. Usa un objeto JSON

prompt = f"""
Identify the following items from the review text: 
- a list of emotions that the writer of the following review is expressing
- Sentiment (positive or negative)

The review is delimited with triple backticks. \
Format your response as a JSON object with \
"Emotion" and "Sentiment" as the keys.
Make your response as short as possible.

Review text: '''{review}'''
"""
response = get_chat_completion(prompt)
print(response)

{
    "Emotion": ["dignified", "appropriate", "gratitude", "uneven mixtape", "low-key", "acknowledgement of the pandemic", 
                "jubilant Paris segment", "clichÃ©-filled speech","strangeness"],
    "Sentiment": ["positive","positive","positive","negative","negative","negative",
                  "positive","negative","mixed"]
}


#### Desafío para Estudiantes:
Deduce 3 temas (tópicos) de la historia a continuación.

In [22]:
story = """
With the floor being just millimeters off the ground, combined with the undulation and inconsistencies of the track surface, sparking is almost a given.
Different factors affect the amount of sparking from an F1 car, with the most common being the fuel load. In qualifying, cars run low fuel for optimum weight and higher speeds, but on race day cars can have up to 110kg of fuel to last them the entire grand prix.
You may have noticed planks of wood on the floor of Formula 1 cars, on the rare occasion that the underside is visible. The purpose of this ‘skid block’ is to ensure that all cars are following the ride height regulations. If the wooden plank has been worn down by more than a millimeter at the end of a race, that car will be disqualified.
But, as expected, teams take advantage of a helpful loophole when it comes to ride height. They place pieces of titanium in strategic parts of the floor, which are designed to strike the track instead of the skid block, thus reducing the amount of wear on the plank and producing a spectacular shower of sparks.
F1 cars are piloted by the world’s fastest and most talented drivers, earning millions of dollars a year; they’ve got to earn their paycheck somehow! F1 cars do not have traction control or an Anti-lock Braking System (ABS), making them more of a challenge to drive.
Occasionally, you might see drivers plow straight into a corner amidst a puff of smoke, with their car seemingly refusing to turn into the corner. This is called a lockup and is a direct result of not having ABS. If a driver applies too much pressure on the brakes too quickly, they will momentarily lock and drag the tyres along the track surface, leaving a flat spot of worn rubber which then causes uncomfortable vibrations for the driver.
If a driver seems to lose control at the exit of a low-speed corner and spin the rear tyres, it’s because they’re in full control of the car’s traction or sometimes the lack thereof. Drivers are just as cautious on the gas pedal as they are on the brakes, which they nurse to their correct operating temperature and use to their maximum efficiency.
"""

In [23]:
# Deduce 3 temas (tópicos) de la historia a continuación.

prompt = f"""
Deduce 3 temas tópicos de la historia delimitada entre comillas \


Historia: '''{story}'''
"""
response = get_chat_completion(prompt)
print(response)

1. Mechanics and technical aspects of Formula 1 cars, including ride height regulations, skid blocks, and sparking.
2. The challenges and skills required for driving an F1 car, such as the lack of traction control and ABS.
3. The importance of fuel load in F1 races and its impact on speed and performance.


#### Táctica 4: Transformación: especifica el idioma objetivo y el estilo de escritura, y solicita una revisión gramatical.

#### Tarea para Estudiantes:
Traductor Universal

Personas de todo el mundo desean conocer las noticias de los Juegos Olímpicos en su idioma nativo. En este caso, las noticias deben ser traducidas a diferentes idiomas. Traduce cada elemento de noticias a continuación tanto al coreano como al inglés.

In [24]:
news = [
"Palestine received a universality invitation from FINA to send two top-ranked swimmers (one per gender) in their respective individual events to the Olympics, \
based on the FINA Points System of June 28, 2021.",
"比赛包括两轮：预赛和决赛。预赛成绩最好的8次接力队晋级决赛。必要时使用游泳比赛来打破平局以晋级下一轮。"]

In [25]:
# Escribe aquí el código para la tarea .

for n in news:
    prompt = f"""
    Your task is to translate news into Korean and English.\

    The news is provided below, delimited by triple backticks. \
    
    Show the result in an array, indicating the language of each translation.\
    
    News: ```{n}```
    """

    response = get_chat_completion(prompt)
    print(response)

[
  {
    language: "Korean",
    translation: "팔레스타인은 2021년 6월 28일 FINA 포인트 시스템을 기반으로, 각 성별 별 개인 종목에서 상위 랭킹의 수영 선수 두 명을 올림픽에 파견하기 위해 FINA로부터 보편성 초청장을 받았습니다."
  },
  {
    language: "English",
    translation: "Palestine has received a universality invitation from FINA to send two top-ranked swimmers (one per gender) in their respective individual events to the Olympics, based on the FINA Points System of June 28, 2021."
  }
]
[
  {
    language: "Korean",
    translation: "경기는 예선전과 결승전으로 구성됩니다. 예선전에서 최고 성적을 거둔 8개의 계주 팀이 결승전에 진출합니다. 필요한 경우 수영 경기를 사용하여 동점을 깨고 다음 라운드에 진출할 수 있습니다."
  },
  {
    language: "English",
    translation: "The competition consists of two rounds: preliminaries and finals. The top eight relay teams with the best scores in the preliminaries advance to the finals. If necessary, swimming competitions are used to break ties and advance to the next round."
  }
]


#### Tarea para Estudiantes:
Transformación de Tono

La escritura puede variar según la audiencia objetivo. ChatGPT puede producir diferentes tonos. Transforma el siguiente mensaje en una carta de negocios.

In [26]:
original_message = "David, it's John! OMG, the Olympic game is so crazy"

In [27]:
# Escribe aquí el código para la tarea .

prompt = f"""
Your task is to write a formal business letter of the message below, \
which is delimited by triple backticks. \


'''{original_message}'''
"""
response = get_chat_completion(prompt)
print(response)

[Your Name]
[Your Address]
[City, State ZIP Code]
[Date]

David [Last Name]
[Recipient's Address]
[City, State ZIP Code]

Dear Mr. [Last Name],

I hope this letter finds you well. I am writing to you today to express my excitement about the Olympic games that are currently taking place.

As a fellow sports enthusiast, I cannot help but be amazed by the incredible performances of the athletes from all over the world. The level of competition is truly inspiring and it is a testament to what hard work and dedication can achieve.

I wanted to take a moment to share my enthusiasm with you as I know that we both share a passion for sports. It is always great to hear from like-minded individuals who appreciate the beauty of athletic achievement.

Thank you for taking the time to read this letter and I look forward to hearing your thoughts on this year's Olympic games.

Sincerely,

[Your Signature]

[Your Typed Name]


#### Tarea para Estudiantes:
Conversión de Formato

ChatGPT puede traducir entre formatos. El prompt debe describir los formatos de entrada y salida. Convierte los siguientes datos JSON al formato HTML.

In [28]:
data_json = { "The 2020 Summer Olympics Opening Ceremony audience name list" :[ 
    {"name":"Shyam", "email":"shyamjaiswal@gmail.com"},
    {"name":"Bob", "email":"bob32@gmail.com"},
    {"name":"Jai", "email":"jai87@gmail.com"}
]}

In [29]:
# Escribe aquí el código para la tarea .

prompt = f"""
Tu tarea es identificar el formato de los datos del texto entre asteriscos
y convertirlos a HTML table with column headers and title


***{data_json}***
"""
response = get_chat_completion(prompt)
print(response)

<table>
  <caption>The 2020 Summer Olympics Opening Ceremony audience name list</caption>
  <thead>
    <tr>
      <th>name</th>
      <th>email</th>
    </tr>
  </thead>
  <tbody>
    <tr><td>Shyam</td><td>shyamjaiswal@gmail.com</td></tr>
    <tr><td>Bob</td><td>bob32@gmail.com</td></tr>
    <tr><td>Jai</td><td>jai87@gmail.com</td></tr>
  </tbody>  
</table>


#### Tarea para Estudiantes:
Revisión ortográfica y gramatical del siguiente texto.

Para indicarle al modelo LLM que deseas que revise tu texto, instruye al modelo para que 'revise' o 'revise y corrija'.

In [30]:
check_text = [ 
  "Alongside the main Emblem blue, the five other colors use in the branding of the 2020 Games is : Kurenai red,   Ai blue,   Sakula pink,   Fuji purple, and   Matsuba green.",
  "The competition have three round: heats, semifinals, and a final."
]

In [31]:
# Escribe aquí el código para la tarea .

prompt = f"""
Tu tarea es hacer revisar y corregir el texto entre comillas triples.
No traduzcas el contenido, la revisión y correción se debe realizar en ingles.
Indica si se hizo alguna corrección o si el texto original esta correcto
Luego de mostrar la correccion, traduce el texto a español


***{check_text}***
"""
response = get_chat_completion(prompt)
print(response)

"Alongside the main emblem blue, the five other colors used in the branding of the 2020 Games are: Kurenai red, Ai blue, Sakula pink, Fuji purple, and Matsuba green."
- Corrección: "use" se cambia a "used".

"The competition has three rounds: heats, semifinals, and a final."
- El texto original está correcto.

Traducción:
"Junto al azul del emblema principal, los otros cinco colores utilizados en la marca de los Juegos de 2020 son: rojo Kurenai, azul Ai, rosa Sakula, púrpura Fuji y verde Matsuba."
"La competencia tiene tres rondas: eliminatorias (heats), semifinales y una final."


#### Táctica 5: Ampliación: personaliza la respuesta automática

#### Tarea para Estudiantes:
Personaliza la respuesta automática al siguiente correo electrónico de un cliente.

El cliente tuvo un problema al comprar un boleto para los Juegos Olímpicos.

In [32]:
# given the sentiment from the tactic on "inferring",
# and the original customer message, customize the email
#sentiment = "positive"

# review for a ticket transaction
review = f"""
I bought the ticket of "Men's 100 metre freestyle swimming" game last week.\
The transaction went through successfully. 
"""



In [33]:
# Escribe aquí el código para la tarea .

prompt = f"""
Eres un Asistente de Inteligencia en Atención al Cliente, tu trabajo es revisar los comentarios de los clientes
que compran nuestros servicios de venta de Boletería de Eventos Culturales y Deportivos y emitir un correo electronico de respuesta a cada cliente en función de sus comentarios.
Si el comentario del cliente es de sentimiento positivo o neutral, agradecele la preferencia e invitale a usar otros de nuestros servicios y eventos proximos.
Si el comentario del cliente es de sentimiento negativo, solicita todo aquello que consideres necesario para solucionar su inconveniente.


El comentario es {review}
"""
response = get_chat_completion(prompt)
print(response)

Estimado/a cliente,

Gracias por su comentario. Nos alegra saber que su transacción para la compra de la boletería del evento "Men's 100 metre freestyle swimming" fue exitosa. Esperamos que disfrute del evento y de nuestros servicios en el futuro.

Atentamente,
Asistente de Inteligencia en Atención al Cliente


#### Táctica 6: Chatbot: personalizar conversaciones para tareas o comportamientos específicos

#### Tarea para Estudiantes:
Crear una conversación con el chatbot para saber dónde se celebraron los Juegos Olímpicos de Verano de 2020.

In [34]:
# Escribe aquí el código para la tarea .

def get_completion_from_messages(messages, model=chat_model, temperature=0.45):
    response = openai.ChatCompletion.create(
        engine=model,
        messages=messages,
        temperature=temperature
    )

    return response.choices[0].message["content"]
messages =  [  
{'role':'system', 'content':'Eres un ChatBot Deportivo.'},  
{'role':'user', 'content':'Soy Luis Antonio.'},
{'role':'assistant', 'content': "Hola encantado de saludarte. Qué necesitas saber hoy?"},  
{'role':'user', 'content':'Cuantas veces se han celebrado los JJOO en Japón?'}  ]
response = get_completion_from_messages(messages)
print(response)

Los Juegos Olímpicos de Tokio 2020 serán los segundos que se celebren en Japón. La primera vez que Japón fue sede de los Juegos Olímpicos fue en Tokio en 1964.


### 2.2 Dale tiempo al modelo para "pensar"

#### Táctica 1: Especificar los pasos necesarios para completar una tarea
A veces puedes ayudar al modelo a "ralentizarse" y ofrecer respuestas más robustas y detalladas especificando los pasos que debe seguir.

Pidamos la salida en varios formatos especificados.

In [35]:
prompt = f"""
Your task is to help a journalist summarize information from the article for publication.

Write a title based on the information provided in the context delimited by triple backticks. 
The title should be short, catchy, and reflective of the article's narrative.

After the title, generate five keywords from the context.

After the keywords, include a table to organize the information. 
The table should have two columns. In the first column is the title.
In the second column include the keywords as a list.

Give the table the title 'Article Publishing Information'.

Format everything as HTML that can be used in a website.
Place the title in a <div> element.

All response must be shown in Spanish

Context: ```{story}

""" 

get_chat_completion(prompt)

'Title: ¿Por qué los coches de Fórmula 1 producen chispas?\n\nKeywords:\n- Fórmula 1\n- chispas\n- skid block\n- ABS\n- neumáticos\n\n<table>\n  <caption>Article Publishing Information</caption>\n  <tr>\n    <th>Título</th>\n    <th>Palabras clave</th>\n  </tr>\n  <tr>\n    <td>¿Por qué los coches de Fórmula 1 producen chispas?</td>\n    <td>Fórmula 1, chispas, skid block, ABS, neumáticos</td>\n  </tr>  \n</table>'

#### Táctica 2: Instruir al modelo para que desarrolle su propia solución

Existen dos métodos principales que demostraremos en esta sección para lograr que el modelo resuelva un problema en lugar de apresurarse a una conclusión: cadena de pensamiento (chain-of-thought) y encadenamiento (chaining). Estas estrategias pueden llevar a una mayor precisión, detalle y la capacidad de resolver desafíos complejos.


2.1 - Prompting de cadena de pensamiento (Chain-of-thought)
- Pedir al modelo que razone
- Ejemplo de un solo disparo (one-shot)
- Razonamiento del chatbot

2.2 - Encadenamiento (Chaining)

Continuemos trabajando con el conjunto de datos de los Juegos Olímpicos.

#### 2.2.1 Prompting de Cadena de Pensamiento (Chain-of-Thought Prompting)

Hagamos un poco de matemáticas. Los modelos de GPT ocasionalmente no resuelven muy bien problemas matemáticos directos, así que guiemos al modelo a través del problema.

Descomponemos las tareas en piezas más pequeñas.

Lee más sobre los métodos y la investigación en el documento técnico: https://github.com/openai/openai-cookbook/blob/main/techniques_to_improve_reliability.md

#### Táctica 1: Puedes comenzar pidiéndole específicamente al modelo que piense paso a paso.

In [36]:
text = f"""
    The 2020 Summer Olympics, officially the Games of the XXXII Olympiad and also known as Tokyo 2020, was 
    an international multi-sport event held from 23 July to 8 August 2021 in Tokyo, Japan, with some preliminary 
    events that began on 21 July 2021. Tokyo was selected as the host city during the 125th IOC Session in Buenos Aires, Argentina, 
    on 7 September 2013.Originally scheduled to take place from 24 July to 9 August 2020, the event was postponed to 2021
    on 24 March 2020 due to the global COVID-19 pandemic, the first such instance in the history of the Olympic Games 
    (previous games had been cancelled but not rescheduled). However, the event retained the Tokyo 2020 branding for 
    marketing purposes. It was largely held behind closed doors with no public spectators permitted due to the declaration 
    of a state of emergency in the Greater Tokyo Area in response to the pandemic, the first and only Olympic Games to be 
    held without official spectators. The Games were the most expensive ever, with total spending of over $20 billion.The 
    Games were the fourth Olympic Games to be held in Japan, following the 1964 Summer Olympics (Tokyo), 1972 Winter Olympics 
    (Sapporo), and 1998 Winter Olympics (Nagano). Tokyo became the first city in Asia to hold the Summer Olympic Games twice. 
    The 2020 Games were the second of three consecutive Olympics to be held in East Asia, following the 2018 Winter Olympics 
    in Pyeongchang, South Korea and preceding the 2022 Winter Olympics in Beijing, China. Due to the one-year postponement, 
    Tokyo 2020 was the first and only Olympic Games to have been held in an odd-numbered year and the first Summer Olympics 
    since 1900 to be held in a non-leap year.\nNew events were introduced in existing sports, including 3x3 basketball, 
    freestyle BMX and mixed gender team events in a number of existing sports, as well as the return of madison cycling for 
    men and an introduction of the same event for women. New IOC policies also allowed the host organizing committee to add 
    new sports to the Olympic program for just one Games. The disciplines added by the Japanese Olympic Committee were baseball 
    and softball, karate, sport climbing, surfing and skateboarding, the last four of which made their Olympic debuts, and the 
    last three of which will remain on the Olympic program.The United States topped the medal count by both total golds (39) and 
    total medals (113), with China finishing second by both respects (38 and 89). Host nation Japan finished third, setting a 
    record for the most gold medals and total medals ever won by their delegation at an Olympic Games with 27 and 58. Great 
    Britain finished fourth, with a total of 22 gold and 64 medals. The Russian delegation competing as the ROC finished fifth 
    with 20 gold medals and third in the overall medal count, with 71 medals. Bermuda, the Philippines and Qatar won their 
    first-ever Olympic gold medals. Burkina Faso, San Marino and Turkmenistan also won their first-ever Olympic medals.'
"""

# From Azure documentation
prompt = "¿Quién fue el atleta individual más condecorado (con el máximo de medallas) en los Juegos Olímpicos celebrados en Sydney? realiza un paso a paso para llegar a la respuesta, cita fuentes y ofrece un razonamiento antes de dar la respuesta en este formato: La respuesta es: <nombre>. Coloca cada paso en un salto de linea. No respondas en HTML ni muestres etiquetas"
get_chat_completion(prompt)

'1. Buscar información sobre los Juegos Olímpicos de Sydney en 2000.\n2. Identificar los deportes y disciplinas que se disputaron en esos juegos.\n3. Revisar las medallas otorgadas en cada disciplina para determinar quiénes obtuvieron el máximo número de medallas.\n4. Comparar los resultados de cada atleta para determinar quién fue el más condecorado.\n5. Consultar fuentes confiables como sitios web oficiales del Comité Olímpico Internacional o medios deportivos especializados.\n\nLa respuesta es: Michael Phelps, quien ganó un total de 8 medallas (6 oros y 2 bronces) en natación durante los Juegos Olímpicos de Sydney 2000.'

In [37]:
# Another example
prompt = f"""
What is the largest time zone difference is between the top two countries who 
won the most gold medals in the 2020 Tokyo olympics?

Use the context below and think aloud as you solve the problem, step-by-step.

Context: {text}
"""
get_chat_completion(prompt)

"To find the largest time zone difference between the top two countries who won the most gold medals in the 2020 Tokyo Olympics, we need to first identify which two countries those are. According to the context, the United States topped the medal count by both total golds (39) and total medals (113), with China finishing second by both respects (38 and 89). Therefore, we will be comparing the time zones of these two countries.\n\nThe United States has multiple time zones, but for simplicity's sake, we can use Eastern Standard Time (EST) as a reference point since it is one of the most populous and widely used time zones in America. EST is 13 hours behind Japan Standard Time (JST), which was used during the Tokyo Olympics.\n\nChina also has multiple time zones, but again for simplicity's sake, we can use Beijing Time as a reference point since it was used during the Beijing Olympics in 2008 and is widely recognized as China's standard time zone."

#### Táctica 2: Ejemplo de un solo disparo (One-shot)
Otra táctica común es proporcionar un ejemplo de una consulta y una respuesta ideal. El modelo aprenderá de ese ejemplo y aplicará los patrones a una nueva pregunta.

In [38]:
# Notice how this response may not be ideal, or the most accurate.
prompt = f"""
    The United States has 113 total medals, 39 of which are gold medals. 
    
    Great Britain has 64 medals and 22 gold medals. 
    
    How many more silver and bronze medals does the United States have over Great Britain?
"""

get_chat_completion(prompt)

'The United States has 74 medals that are not gold (113 total medals - 39 gold medals). \n\nGreat Britain has 42 medals that are not gold (64 total medals - 22 gold medals). \n\nTherefore, the United States has 32 more non-gold medals than Great Britain.'

In [39]:
# Give the model a one-shot example to solve the question more thoroughly

prompt = f"""
Question: The United States has 113 total medals, 39 of which are gold medals. How many medals are silver or bronze?
Answer:
[Step 1] - There are three types of medals: gold, silver, and bronze
[Step 2] - We know the gold and total counts of medals, so the number of silver and bronze medals is the difference between the gold (39) and total (113) counts. 
            113-39=74, so there are 74 silver and bronze medals combined. The answer is 74.

===

Answer the following question using similar steps above.

Question: China has 89 total medals, 38 of which are gold medals. How many silver and bronze medals do they have?
Answer:
"""

get_chat_completion(prompt)

'[Step 1] - There are three types of medals: gold, silver, and bronze\n[Step 2] - We know the gold and total counts of medals, so the number of silver and bronze medals is the difference between the gold (38) and total (89) counts. \n            89-38=51, so there are 51 silver and bronze medals combined. The answer is 51.'

#### Táctica 3: Razonamiento de cadena de pensamiento del chatbot (chain-of-thought reasoning)
Puedes incorporar lógica utilizando variables para que el chatbot pueda demostrar de manera dinámica formas específicas de pensar sobre un problema.

**Para ingresar la información, aparecerá un cuadro de texto en la parte superior de la pantalla. Deberías ver una ventana emergente. Escribe 'quit' si quieres salir.**

In [40]:
# Ask the bot to help you make a decision such as deciding whether to take a job or choose between restaurants.
# If the model does not immediately respond to your query, wait 1-5 seconds and retype it.
# If it is not allowing you to give an input, restart the kernel in the navigation bar.
# Type "quit" to end the session

context = '''
        You are a decision bot. Your job is to help come to decision by asking a series of questions one at a time and coming to a reasonable decision based on the information provided.

        You will use the following format to help create the series of questions.

        Template: 
        [Problem/Scenario/Question]: [Provide a brief description of the problem, scenario, or question.]

        Chain of thought:

        [Step 1]: Identify the [key element/variable] in the [problem/scenario/question].
        [Step 2]: Understand the [relationship/connection] between [element A] and [element B].
        [Step 3]: [Analyze/Evaluate/Consider] the [context/implication] of the [relationship/connection] between [element A] and [element B].
        [Step 4]: [Conclude/Decide/Determine] the [outcome/solution] based on the [analysis/evaluation/consideration] of [element A], [element B], and their [relationship/connection].
        [Answer/Conclusion/Recommendation]: [Provide a coherent and logical response based on the chain of thought.]

        You will guide the user though a series of questions one at a time. The first question is broad, and they subsequent questions become more specific. 

        Begin by introducing yourself and asking the first question (step 1) only and nothing else, in simple and easy way.
        '''

conversation=[{"role": "system", "content": context}]

while(True):
    print(len(conversation))
    if len(conversation) == 1:
        response = get_completion_from_messages(conversation)
        conversation.append({"role": "assistant", "content": response})
        print("\n" + response + "\n")
    
    user_input = input('Enter your response: ')
    if user_input.lower() == "quit":
        break 
    conversation.append({"role": "user", "content": user_input})
    
    response = get_completion_from_messages(conversation)
    conversation.append({"role": "assistant", "content": response})
    print("\n" + response + "\n")


1

Hello! I am a decision bot. What decision do you need help with today?


Problem/Scenario/Question: Should I go to the beach or the mountains?

Step 1: Identify the key element/variable in the problem/scenario/question.
What are the factors that will help you decide between going to the beach or the mountains?

4

Step 2: Understand the relationship/connection between the climate and the decision to go to the beach or mountains.
How does the climate affect your decision between going to the beach or the mountains?

6

Step 3: Analyze/Evaluate/Consider the context/implication of the relationship/connection between the climate and the decision to go to the beach or mountains.
Considering that you prefer to go to the beach when it's sunny and to the mountains when it's not, what other factors might influence your decision? For example, are you looking for a relaxing or an adventurous trip? Do you prefer to be surrounded by nature or by people?

8

Step 4: Conclude/Decide/Determine the 

#### 2.2.2 - Encadenamiento / Chaining
Similar a algunos ejemplos anteriores, puedes usar las salidas del modelo de consultas anteriores en otras consultas. Más adelante en el Hack te mostraremos cómo hacer esto a escala.

In [41]:
# Extract medal counts for each country from the news article
# Write an article about the summarized information
# Provide a title for the summary

text = f"""
    The 2020 Summer Olympics, officially the Games of the XXXII Olympiad and also known as Tokyo 2020, was an international multi-sport event held from 23 July to 8 August 2021 in Tokyo, Japan, with some preliminary events that began on 21 July 2021. Tokyo was selected as the host city during the 125th IOC Session in Buenos Aires, Argentina, on 7 September 2013.Originally scheduled to take place from 24 July to 9 August 2020, the event was postponed to 2021 on 24 March 2020 due to the global COVID-19 pandemic, the first such instance in the history of the Olympic Games (previous games had been cancelled but not rescheduled). However, the event retained the Tokyo 2020 branding for marketing purposes. It was largely held behind closed doors with no public spectators permitted due to the declaration of a state of emergency in the Greater Tokyo Area in response to the pandemic, the first and only Olympic Games to be held without official spectators. The Games were the most expensive ever, with total spending of over $20 billion.The Games were the fourth Olympic Games to be held in Japan, following the 1964 Summer Olympics (Tokyo), 1972 Winter Olympics (Sapporo), and 1998 Winter Olympics (Nagano). Tokyo became the first city in Asia to hold the Summer Olympic Games twice. The 2020 Games were the second of three consecutive Olympics to be held in East Asia, following the 2018 Winter Olympics in Pyeongchang, South Korea and preceding the 2022 Winter Olympics in Beijing, China. Due to the one-year postponement, Tokyo 2020 was the first and only Olympic Games to have been held in an odd-numbered year and the first Summer Olympics since 1900 to be held in a non-leap year.\nNew events were introduced in existing sports, including 3x3 basketball, freestyle BMX and mixed gender team events in a number of existing sports, as well as the return of madison cycling for men and an introduction of the same event for women. New IOC policies also allowed the host organizing committee to add new sports to the Olympic program for just one Games. The disciplines added by the Japanese Olympic Committee were baseball and softball, karate, sport climbing, surfing and skateboarding, the last four of which made their Olympic debuts, and the last three of which will remain on the Olympic program.The United States topped the medal count by both total golds (39) and total medals (113), with China finishing second by both respects (38 and 89). Host nation Japan finished third, setting a record for the most gold medals and total medals ever won by their delegation at an Olympic Games with 27 and 58. Great Britain finished fourth, with a total of 22 gold and 64 medals. The Russian delegation competing as the ROC finished fifth with 20 gold medals and third in the overall medal count, with 71 medals. Bermuda, the Philippines and Qatar won their first-ever Olympic gold medals. Burkina Faso, San Marino and Turkmenistan also won their first-ever Olympic medals.'
"""

prompt = f"""
    Based on the context below, write a JSON object that contains the number of gold and total medals for each country.
    Context: {text}
"""
num_medals_dict = get_chat_completion(prompt)

prompt = f"""
    Write a brief article about the winners and losers of the Olympics based on medal count:
    {num_medals_dict}
"""
summary = get_chat_completion(prompt)
print(summary)

prompt = f"""
    Give the summary a title in 5 words:
    {summary}
"""
title = get_chat_completion(prompt)
print(title)

The 2021 Olympics in Tokyo have come to an end, and the medal count has been tallied. The United States came out on top with a total of 113 medals, including 39 gold medals. China was a close second with 89 total medals, including 38 golds. Japan finished third with 58 total medals, including an impressive haul of 27 golds.

Great Britain and the ROC (Russian Olympic Committee) rounded out the top five with 64 and 71 total medals respectively. However, there were also some surprise winners in this year's Olympics. Bermuda won its first-ever Olympic gold medal thanks to triathlete Flora Duffy's incredible performance.

The Philippines also had reason to celebrate as weightlifter Hidilyn Diaz won their first-ever Olympic gold medal in any sport. Qatar also made history by winning its first-ever Olympic gold medal thanks to high jumper Mutaz Essa Barshim.

On the other hand, there were some countries that did not perform
"US tops medal count in 2021 Olympics"


#### Tarea para Estudiantes (Encadenamiento / Chaining):

Tu tarea ahora es escribir código que determine el país que ganó la mayor cantidad de medallas de plata y bronce combinadas.

Podemos ver que el modelo tiene un rendimiento deficiente al responder la pregunta directamente.

In [42]:
prompt = f"""
    Based on the context, which country had the most silver and bronze medals?
    Context: {text}
"""

get_chat_completion(prompt)

'The context does not provide information on which country had the most silver and bronze medals.'

Escribe prompts en la celda a continuación que ayudarán al modelo a responder la pregunta desglosando las tareas en diferentes pasos. Asegúrate de que responda solo con la información que se le proporcionó. Este concepto de fundamentación se presentará con más detalle en el Reto 3.

Deberías poder lograr que el modelo responda la pregunta en 2-3 pasos.

In [44]:
# Escribe aquí el código para la tarea .
prompt = f"""
    Basado en el contexto otorgado, escribe un objeto JSON que contenga el numero de 
    Medallas de Oro y el numero de Medallas Total conseguidas por cada país.
    Responde en español.
    Context: {text}
"""
medallero1 = get_chat_completion(prompt)
print(medallero1)

prompt =f"""
    Calcula el total de Medallas Plata - Bronce conseguidas por cada país
    
    Escribe un nuevo objeto JSON con dos claves: Pais y Combinado Plata-Bronce
    {medallero1}
"""
medallero_pl_br = get_chat_completion(prompt)
print(medallero_pl_br)

prompt = f"""
    Basado en el texto dado, indica cual fue el razonamiento realizado para llegar a la respuesta y 
    dime ¿cual país obtuvo la mayor combinación de medallas de plata y bronce? Escribe el país en mayúsculas
    
    {medallero_pl_br}
"""
respuesta = get_chat_completion(prompt)
print(respuesta)

{
   "Estados Unidos": {
      "Medallas de Oro": 39,
      "Medallas Totales": 113
   },
   "China": {
      "Medallas de Oro": 38,
      "Medallas Totales": 89
   },
   "Japón": {
      "Medallas de Oro": 27,
      "Medallas Totales": 58
   },
   "Gran Bretaña":{
       "Medallas de Oro" :22, 
       Medallas Totales:64
    },
    ROC:{
        Medallas de Oro:20, 
        Medalla Total:71
    }, 
    Bermuda:{
        Medalla de oro:1, 
        Medalla Total:1
    }, 
    Filipinas:{
        Medalla de oro:1, 
        Medalla Total:2},
     Qatar:{
         Medalla De oro :1,
         medall total :2},
     Burkina Faso
{
   "Estados Unidos": {
      "Combinado Plata-Bronce": 74
   },
   "China": {
      "Combinado Plata-Bronce": 51
   },
   "Japón": {
      "Combinado Plata-Bronce": 31
   },
   "Gran Bretaña":{
       "Combinado Plata-Bronce" :42 
    },
    ROC:{
        "Combinado Plata-Bronce":51 
    }, 
    Bermuda:{
        "Combinado Plata-Bronce":0 
    }, 
    Filipinas:{


## Conclusión

En este primer desafío, cubrimos cómo interactuar con Azure OpenAI para diferentes objetivos. Esperamos que hayas podido ver la versatilidad dinámica de los modelos y cómo pueden usarse para resolver una variedad de problemas utilizando diferentes técnicas.

Le dimos a la API piezas cortas de texto usando variables fijas. En el próximo conjunto de desafíos, verás cómo usar la API con conjuntos de datos más grandes.